In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-27 20:29:38--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.38MB/s    in 0.2s    

2021-04-27 20:29:38 (4.38 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
apparel_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
apparel_df.show(15)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [ ]:
# Drop duplicates and null values 
print(apparel_df.count())
apparel_df = apparel_df.dropna()
print(apparel_df.count())
apparel_df = apparel_df.dropDuplicates()
print(apparel_df.count())

5906333
5905269
5905269


In [ ]:
# View the Schema
apparel_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Create new data frames for tables in Schema
review_id_table = apparel_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(15)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000LG8H70CNQ|     696306|B00O3A0RHY|     246430568| 2015-01-25|
|R100BCEUAIHZQC|   11807812|B00CASTERA|     600575877| 2014-06-03|
|R100EOB2GC5UYQ|   15017601|B002MLE36W|     988639002| 2014-10-26|
|R100MYSZIXWX1D|   44291413|B007HSEDVK|        174345| 2012-10-31|
|R100UMGHMFVXKC|   24539177|B00F52CUCE|     162326772| 2014-03-18|
|R101CJDLDDM19P|   10041263|B00NIX31K2|     559380483| 2014-12-16|
|R101OE6ZEL1A52|   20521864|B00CB1S40E|     248821115| 2014-02-08|
| R101PL938YJV0|   44457530|B00AF7D7VC|     656221590| 2013-11-14|
|R101QMSD2TQTAP|   14103889|B00N9KGZWA|     906968721| 2015-04-30|
|R101V43VD0FQ5Y|   23446176|B007WPF25E|     619910868| 2013-07-01|
|R102HAEXYCF1YM|   20169548|B00DECZ2KE|     913386738| 2014-02-09|
|R102KCQDXFRN3E|   18522476|B000092MPB|     673568662| 2006-06

In [ ]:
products = apparel_df.select(["product_id", "product_title"])
products.show(15)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00O3A0RHY|Giant Panda Face ...|
|B00CASTERA|  Rothco Jungle Hats|
|B002MLE36W|Leg Avenue Women'...|
|B007HSEDVK|LAST KISS Ombré W...|
|B00F52CUCE|Magiftbox Men's S...|
|B00NIX31K2|ROEWELL® 4 Sets o...|
|B00CB1S40E|Valco Baby Snap4 ...|
|B00AF7D7VC|Louis Raphael ROS...|
|B00N9KGZWA|Women`s Push Up T...|
|B007WPF25E|Kangol Men's Josh...|
|B00DECZ2KE|London Times Wome...|
|B000092MPB|      Zip-front suit|
|B00CFET4Y2|Kanu Surf Men's C...|
|B00O3MJRRS|Modern Minute - N...|
|B003N63ET2|Champion Women's ...|
+----------+--------------------+
only showing top 15 rows



In [ ]:
customers = apparel_df.select(["customer_id"])
customers.show(15)

+-----------+
|customer_id|
+-----------+
|     696306|
|   11807812|
|   15017601|
|   44291413|
|   24539177|
|   10041263|
|   20521864|
|   44457530|
|   14103889|
|   23446176|
|   20169548|
|   18522476|
|   14009089|
|   22997491|
|   45763066|
+-----------+
only showing top 15 rows



In [ ]:
customers = customers.groupby(['customer_id']).count()
customers.show(15)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   12955905|    5|
|    3909801|    1|
|   17063052|    1|
|   13245107|    1|
|   15149043|    5|
|    5137716|    1|
|   42673458|   23|
|   15476378|    1|
|   11615457|    1|
|   52209445|    2|
|   47277612|   18|
|     481101|    1|
|    3207957|    2|
|   20874267|    3|
|   21580335|    2|
+-----------+-----+
only showing top 15 rows



In [ ]:
customers = customers.withColumnRenamed("count", "customer_count")
customers.show(15)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12955905|             5|
|    3909801|             1|
|   17063052|             1|
|   13245107|             1|
|   15149043|             5|
|    5137716|             1|
|   42673458|            23|
|   15476378|             1|
|   11615457|             1|
|   52209445|             2|
|   47277612|            18|
|     481101|             1|
|    3207957|             2|
|   20874267|             3|
|   21580335|             2|
+-----------+--------------+
only showing top 15 rows



In [ ]:
vine_table = apparel_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(15)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000LG8H70CNQ|          5|            1|          1|   N|
|R100BCEUAIHZQC|          5|            0|          0|   N|
|R100EOB2GC5UYQ|          4|            0|          0|   N|
|R100MYSZIXWX1D|          2|            0|          0|   N|
|R100UMGHMFVXKC|          5|            0|          1|   N|
|R101CJDLDDM19P|          5|            0|          1|   N|
|R101OE6ZEL1A52|          5|            0|          1|   N|
| R101PL938YJV0|          5|            0|          0|   N|
|R101QMSD2TQTAP|          5|            0|          1|   N|
|R101V43VD0FQ5Y|          4|            1|          1|   N|
|R102HAEXYCF1YM|          5|            0|          0|   N|
|R102KCQDXFRN3E|          2|            1|          1|   N|
|R102QMA01NZA6Z|          5|            0|          0|   N|
|R102S2K1PLWHI0|          4|            

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)